In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [4]:
df=pd.read_csv('data/menu_items.csv')
df.head(5)

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00


In [5]:
## basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   menu_item_id  32 non-null     int64  
 1   item_name     32 non-null     object 
 2   category      32 non-null     object 
 3   price         32 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ KB


In [6]:
df1 = pd.read_csv('data/order_details.csv')
df1.head(5)

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,1/1/23,11:38:36 AM,109.0
1,2,2,1/1/23,11:57:40 AM,108.0
2,3,2,1/1/23,11:57:40 AM,124.0
3,4,2,1/1/23,11:57:40 AM,117.0
4,5,2,1/1/23,11:57:40 AM,129.0


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12234 entries, 0 to 12233
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_details_id  12234 non-null  int64  
 1   order_id          12234 non-null  int64  
 2   order_date        12234 non-null  object 
 3   order_time        12234 non-null  object 
 4   item_id           12097 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 478.0+ KB


In [10]:
import pyodbc

conn = pyodbc.connect(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=DESKTOP-G3STKJF\SQLEXPRESS;"   
    r"DATABASE=resturant_order;"      
    r"Trusted_Connection=yes;"
)

cursor = conn.cursor()
print("Connection successful")


Connection successful


In [13]:
def insert_dataframe_to_sql(df, table_name, conn):
    cursor = conn.cursor()
    
    # Map pandas dtypes to SQL Server types
    def get_sql_type(dtype):
        if dtype == 'int64':
            return 'INT'
        elif dtype == 'float64':
            return 'FLOAT'
        elif dtype == 'object':
            return 'VARCHAR(MAX)'
        elif dtype == 'bool':
            return 'BIT'
        else:
            return 'VARCHAR(MAX)'
    
    # Create table with proper data types
    columns_def = ", ".join([f"[{col}] {get_sql_type(str(df[col].dtype))}" for col in df.columns])
    create_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}; CREATE TABLE {table_name} ({columns_def});"
    cursor.execute(create_query)
    conn.commit()

    # Insert rows with proper null handling
    for index, row in df.iterrows():
        placeholders = ", ".join(["?"] * len(row))
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        # Replace NaN and None with None for SQL NULL
        values = [None if pd.isna(val) else val for val in row]
        cursor.execute(insert_query, tuple(values))
    
    conn.commit()
    print(f"✓ Inserted {len(df)} rows into {table_name}")

In [14]:
insert_dataframe_to_sql(df, "menu_items", conn)
insert_dataframe_to_sql(df1, "order_details", conn)

print("\n✓ All data loaded successfully!")
conn.close()

✓ Inserted 32 rows into menu_items
✓ Inserted 12234 rows into order_details

✓ All data loaded successfully!
